In [26]:
import plotly.express as px
import pandas as pd

In [27]:
from collections import defaultdict

class Graph:
    def __init__(self):
        self.graph = defaultdict(list)  # Adjacency list representation
        self.reverse_graph = defaultdict(list)  # Reverse graph to track parent-child relationships
    
    def add_edge(self, u, v):
        self.graph[u].append(v)  # Directed edge u -> v
        self.reverse_graph[v].append(u)  # Reverse edge v -> u (parent -> child)
    
    def remove_node(self, node):
        # Get all children of the node
        children = self.graph[node]

        # Get all parents of the node (nodes that have edges pointing to it)
        parents = self.reverse_graph[node]

        # Reassign children of the removed node to its parents
        for parent in parents:
            # Remove the original connection from parent to the removed node
            if node in self.graph[parent]:
                self.graph[parent].remove(node)
            # Add all the children to the parent's adjacency list
            self.graph[parent].extend(children)

        # Remove the node from the graph
        if node in self.graph:
            del self.graph[node]  # Delete the node from the graph
        if node in self.reverse_graph:
            del self.reverse_graph[node]  # Delete the node from reverse graph
        
        # Remove the node from other nodes' adjacency lists
        for neighbors in self.graph.values():
            if node in neighbors:
                neighbors.remove(node)
        
        # Also, remove the node from reverse graph's parent-child relationships
        for children in self.reverse_graph.values():
            if node in children:
                children.remove(node)
    
    def enumerate_levels(self):
        levels = {}  # To store levels of each node
        visited = set()

        def dfs(node, current_level):
            visited.add(node)
            levels[node] = current_level  # Assign the current level to the node

            for neighbor in self.graph[node]:
                if neighbor not in visited:
                    dfs(neighbor, current_level + 1)

        # Handle disconnected graph
        for node in list(self.graph.keys()):  # Use list(self.graph.keys()) to avoid RuntimeError
            if node not in visited:
                dfs(node, 0)  # Start DFS for unvisited components

        return levels


# Example Usage
g = Graph()
#g.add_edge(0, 1)
#g.add_edge(1, 2)
#g.add_edge(2, 3)
#g.add_edge(1, 4)
#g.add_edge(4, 5)
#g.add_edge(6, 1)
#g.add_edge(6, 7)
#g.add_edge(7, 8)
#g.add_edge(8, 9)
#g.add_edge(9, 10)

g.add_edge(1, 2)
g.add_edge(0, 2)
g.add_edge(2, 4)
g.add_edge(2, 3)
g.add_edge(4, 5)
g.add_edge(3, 6)

#g.remove_node(2)

levels = g.enumerate_levels()
print("Node Levels:", levels)

Node Levels: {1: 0, 2: 1, 4: 2, 5: 3, 3: 2, 6: 3, 0: 0}


In [28]:
# this is the center
offset = 2
updated_levels = {node: level - offset for node, level in levels.items()}

In [29]:
level_dict = {val: 0 for val in updated_levels.values()}

In [30]:
from datetime import datetime, timedelta
timeline = []
offset = 4
dt = datetime(2025, 1, 26, 00, 00, 00)  # Example datetime
for id in updated_levels:
    start_date = dt + timedelta(hours=updated_levels[id]*offset)
    end_date = dt + timedelta(hours=updated_levels[id]*offset+offset)
    timeline.append(dict(System=level_dict[updated_levels[id]], Entity=f"Entity: {id}", Start=start_date, Finish=end_date, Document=dt))
    level_dict[updated_levels[id]] += 1
    
level_dict = {val: 0 for val in updated_levels.values()}
dt = datetime(2025, 1, 25, 00, 00, 00)  # Example datetime
for id in updated_levels:
    start_date = dt + timedelta(hours=updated_levels[id]*offset)
    end_date = dt + timedelta(hours=updated_levels[id]*offset+offset)
    timeline.append(dict(System=level_dict[updated_levels[id]], Entity=f"Superlong Entity: {id}", Start=start_date, Finish=end_date, Document=dt))
    level_dict[updated_levels[id]] += 1

In [31]:
df = pd.DataFrame(timeline)

fig = px.timeline(df, x_start="Start", x_end="Finish", y="System", color="Entity", text="Entity")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.update_traces(textposition='inside')
fig.update_layout(
    xaxis=dict(
        tickformat="%Y-%m-%d",  # Format the ticks to show only the date,
        dtick="D1"
    )
)
fig.update_yaxes(visible=False, showticklabels=False)  # Hide the y-axis


fig.update_layout(barmode='overlay')
fig.update_traces(hovertemplate="Document %{base|%Y-%m-%d}<br>", 
                  textangle=0, 
                  insidetextfont=dict(size=56))

fig.show()